In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s3e25/sample_submission.csv
/kaggle/input/playground-series-s3e25/train.csv
/kaggle/input/playground-series-s3e25/test.csv


In [1]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import VotingRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor


In [4]:
traindata=pd.read_csv("/kaggle/input/playground-series-s3e25/train.csv")
testdata=pd.read_csv("/kaggle/input/playground-series-s3e25/test.csv")
traindata.head(5)

,id,allelectrons_Total,density_Total,allelectrons_Average,val_e_Average,atomicweight_Average,ionenergy_Average,el_neg_chi_Average,R_vdw_element_Average,R_cov_element_Average,zaratio_Average,density_Average,Hardness
0,0,100.0,0.841611,10.0,4.8,20.612526,11.08810,2.766,1.732,0.860,0.496070,0.91457,6.0
1,1,100.0,7.558488,10.0,4.8,20.298893,12.04083,2.755,1.631,0.910,0.492719,0.71760,6.5
2,2,76.0,8.885992,15.6,5.6,33.739258,12.08630,2.828,1.788,0.864,0.481478,1.50633,2.5
3,3,100.0,8.795296,10.0,4.8,20.213349,10.94850,2.648,1.626,0.936,0.489272,0.78937,6.0
4,4,116.0,9.577996,11.6,4.8,24.988133,11.82448,2.766,1.682,0.896,0.492736,1.86481,6.0


In [5]:
traindata.isna().sum()

id                       0
allelectrons_Total       0
density_Total            0
allelectrons_Average     0
val_e_Average            0
atomicweight_Average     0
ionenergy_Average        0
el_neg_chi_Average       0
R_vdw_element_Average    0
R_cov_element_Average    0
zaratio_Average          0
density_Average          0
Hardness                 0
dtype: int64

In [6]:
traindata.drop(['id'],axis=1)

,allelectrons_Total,density_Total,allelectrons_Average,val_e_Average,atomicweight_Average,ionenergy_Average,el_neg_chi_Average,R_vdw_element_Average,R_cov_element_Average,zaratio_Average,density_Average,Hardness
0,100.0,0.841611,10.000000,4.800000,20.612526,11.088100,2.766000,1.732000,0.860000,0.496070,0.91457,6.0
1,100.0,7.558488,10.000000,4.800000,20.298893,12.040830,2.755000,1.631000,0.910000,0.492719,0.71760,6.5
2,76.0,8.885992,15.600000,5.600000,33.739258,12.086300,2.828000,1.788000,0.864000,0.481478,1.50633,2.5
3,100.0,8.795296,10.000000,4.800000,20.213349,10.948500,2.648000,1.626000,0.936000,0.489272,0.78937,6.0
4,116.0,9.577996,11.600000,4.800000,24.988133,11.824480,2.766000,1.682000,0.896000,0.492736,1.86481,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...
10402,128.0,7.558488,12.000000,4.000000,26.385218,11.330440,2.644000,1.631000,0.892000,0.496070,1.79607,4.0
10403,30.0,1.743160,10.000000,5.333333,20.766935,14.163933,3.090000,1.556667,0.866667,0.480390,0.81480,5.0
10404,196.0,30.920000,24.500000,5.500000,53.490297,10.074300,2.295000,1.545000,1.120000,0.469715,2.11540,1.8
10405,38.0,1.553160,12.666667,4.666667,26.621687,11.290033,2.743333,1.756667,0.980000,0.486507,0.77755,6.0


In [8]:
X=traindata.drop(['Hardness'],axis=1)
y=traindata['Hardness']

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [11]:
# Function to evaluate and print results
def evaluate_model(y_true, y_pred, model_name):
    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)

    print(f"\n{model_name}:")
    print(f"Mean Squared Error: {mse:.4f}")
    print(f"Mean Absolute Error: {mae:.4f}")
    print(f"R2 Score: {r2:.4f}")

    return {'MSE': mse, 'MAE': mae, 'R2': r2}

In [24]:
# Random Forest Regressor
RandomForestModel = RandomForestRegressor(random_state=42)
RandomForestModel.fit(X_train_scaled, y_train)
RF_y_pred = RandomForestModel.predict(X_test_scaled)
rf_results = evaluate_model(y_test, RF_y_pred, "Random Forest")


Random Forest:
Mean Squared Error: 1.5209
Mean Absolute Error: 0.9198
R2 Score: 0.4405


In [13]:
# Gradient Boosting Regressor
GradientBoostingModel = GradientBoostingRegressor(random_state=42)
GradientBoostingModel.fit(X_train_scaled, y_train)
GB_y_pred = GradientBoostingModel.predict(X_test_scaled)
gb_results = evaluate_model(y_test, GB_y_pred, "Gradient Boosting")


Gradient Boosting:
Mean Squared Error: 1.5042
Mean Absolute Error: 0.9243
R2 Score: 0.4466


In [14]:
# Decision Tree Regressor
DecisionTreeModel = DecisionTreeRegressor(random_state=42)
DecisionTreeModel.fit(X_train_scaled, y_train)
DT_y_pred = DecisionTreeModel.predict(X_test_scaled)
dt_results = evaluate_model(y_test, DT_y_pred, "Decision Tree")


Decision Tree:
Mean Squared Error: 3.1739
Mean Absolute Error: 1.2447
R2 Score: -0.1677


In [15]:
# K-Nearest Neighbors Regressor
KNNModel = KNeighborsRegressor()
KNNModel.fit(X_train_scaled, y_train)
KNN_y_pred = KNNModel.predict(X_test_scaled)
knn_results = evaluate_model(y_test, KNN_y_pred, "KNN")


KNN:
Mean Squared Error: 1.9848
Mean Absolute Error: 1.0514
R2 Score: 0.2698


In [16]:
# XGBoost Regressor
XGBoostModel = XGBRegressor(random_state=42)
XGBoostModel.fit(X_train_scaled, y_train)
XGB_y_pred = XGBoostModel.predict(X_test_scaled)
xgb_results = evaluate_model(y_test, XGB_y_pred, "XGBoost")


XGBoost:
Mean Squared Error: 1.6229
Mean Absolute Error: 0.9524
R2 Score: 0.4029


In [17]:
# CatBoost Regressor
CatBoostModel = CatBoostRegressor(random_state=42, verbose=0)
CatBoostModel.fit(X_train_scaled, y_train)
CatBoost_y_pred = CatBoostModel.predict(X_test_scaled)
catboost_results = evaluate_model(y_test, CatBoost_y_pred, "CatBoost")



CatBoost:
Mean Squared Error: 1.4876
Mean Absolute Error: 0.9097
R2 Score: 0.4527


In [18]:
# Voting Regressor
VotingModel = VotingRegressor([
    ('rf', RandomForestModel),
    ('gb', GradientBoostingModel),
    ('dt', DecisionTreeModel),
    ('knn', KNNModel),
    ('xgb', XGBoostModel),
    ('catboost', CatBoostModel)
])
VotingModel.fit(X_train_scaled, y_train)
Voting_y_pred = VotingModel.predict(X_test_scaled)
voting_results = evaluate_model(y_test, Voting_y_pred, "Voting Regressor")


Voting Regressor:
Mean Squared Error: 1.5364
Mean Absolute Error: 0.9245
R2 Score: 0.4347


In [20]:
test_predictions = CatBoostModel.predict(testdata)
# Create submission file
submission = pd.DataFrame({'id': testdata['id'], 'Hardness': test_predictions})
submission.to_csv('submission.csv', index=False)
print("\nSubmission file created successfully!")


Submission file created successfully!
